In [56]:
import numpy as np
import scipy as sp

In [57]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [58]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

In [59]:
from dataclasses import dataclass, field

@dataclass
class Params:
    mass: float = 1.352  # [kg]
    inertia: np.ndarray = field(default_factory=lambda: np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ]))
    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches
    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient
    arm_length: float = 0.3814 / 2.0  # [m] Arm length
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)

p = Params()

### Finishing the controller
In this notebook, you'll take your previous implementation where we just controller altitude (z-position) and expand it to build the full position-attitude loop.

**Note that I updated the controller params class with some extra parameters!**

#### Recommended steps
- Copy your 3D quadrotor model
- Copy your controller implementation
- Expand the controller class
    - Compute the desired roll/pitch using the equations from the lecture
    - Compute the attitude errors
    - Compute the desired angular acceleration
    - Compute `u_2`
    - Tune your system (the gains provided _should_ provide a relatively stable starting point!)

### The dynamics

In [60]:
# Copy your dynamics here!
from quadrotor.dynamics import QuadrotorState
from sym import Rot3

class FullQuadrotorDynamics(QuadrotorDynamicsBase):
    def step(self, t: float, commands: QuadrotorCommands, state: QuadrotorState) -> QuadrotorState:
        dt = 0.005  # Time step

        position, velocity = state.position, state.velocity
        orientation = state.orientation
        angular_velocity = state.angular_velocity

        u1, u2 = commands.u1, commands.u2

        acceleration = (u1 / p.mass) * np.array([0, 0, 1]) - np.array([0, 0, p.g])
        new_velocity = velocity + acceleration * dt
        new_position = position + new_velocity * dt

        angular_acceleration = np.linalg.inv(p.inertia) @ (u2 - np.cross(angular_velocity, p.inertia @ angular_velocity))
        new_angular_velocity = angular_velocity + angular_acceleration * dt
        new_orientation = dRot3(orientation, new_angular_velocity)

        return QuadrotorState(
            position=new_position,
            velocity=new_velocity,
            orientation=new_orientation,
            angular_velocity=new_angular_velocity,
        )


### The controller

In [61]:
@dataclass
class ControllerParams:
    #Includes controller gains
    K_p: np.ndarray = field(default_factory=lambda: np.diag([1, 1, 100]))
    K_d: np.ndarray = field(default_factory=lambda: np.diag([0.5, 0.5, 10]))
    Ka_p: np.ndarray = field(default_factory=lambda: np.diag([300, 300, 50]))
    Ka_d: np.ndarray = field(default_factory=lambda: np.diag([50, 50, 20]))
    rotor_rate_min: float = 180
    rotor_rate_max: float = 600

controller_p = ControllerParams()

In [62]:
# Write your controller here!
from quadrotor.controller import ControllerBase

class Controller(ControllerBase):
    def __init__(self, params: ControllerParams):
        self.params = params

    def step(self, t: float, trajectory: TrajectoryState, state: QuadrotorState) -> QuadrotorCommands:
        desired_position = trajectory.position
        actual_position = state.position
        position_error = desired_position - actual_position

        desired_velocity = np.zeros(3)  # Assume we want to come to a stop at the target
        actual_velocity = state.velocity
        velocity_error = desired_velocity - actual_velocity

        # PD control for thrust (u1)
        commanded_acceleration = self.params.K_p @ position_error + self.params.K_d @ velocity_error
        u1 = p.mass * (p.g + commanded_acceleration[2])

        # Desired roll/pitch
        yaw = state.orientation.to_yaw_pitch_roll()[0]  # Decompose to get yaw angle
        phi_des = (1 / p.g) * (commanded_acceleration[0] * np.sin(yaw) - commanded_acceleration[1] * np.cos(yaw))
        theta_des = (1 / p.g) * (commanded_acceleration[0] * np.cos(yaw) + commanded_acceleration[1] * np.sin(yaw))

        # Attitude control
        desired_orientation = Rot3.from_yaw_pitch_roll(trajectory.yaw, theta_des, phi_des)
        orientation_error = desired_orientation.compose(state.orientation.inverse())

        orientation_error_matrix = orientation_error.to_rotation_matrix()  # Add orientation error matrix
        orientation_error_vec = np.array([orientation_error_matrix[2, 1], 
                                          orientation_error_matrix[0, 2], 
                                          orientation_error_matrix[1, 0]])

        angular_velocity_error = np.zeros(3) - state.angular_velocity

        # PD control for angular velocities (u2)
        u2 = self.params.Ka_p @ orientation_error_vec + self.params.Ka_d @ angular_velocity_error

        # Pack the control commands (fix: pass positional arguments u1 and u2)
        #Now I am getting an error RE I have -1 positional args than I need...idk why this would be
        return QuadrotorCommands(u1, u2)
        

### The trajectory
Just something simple, mess around with it!

In [63]:
#class JumpTrajectory(TrajectoryBase):
    #def eval(self, t: float):
        #x = 0 if t < 1.0 or t > 5.0 else 1.0
        #y = 0 if t < 2.0 or t > 5.0 else 1.0
        #position = np.array([x, y, 0])
        #yaw_rate = 0.5
        #yaw = yaw_rate * t
        #return TrajectoryState(t, position=position, yaw=yaw, yaw_rate=yaw_rate)

class SmoothTrajectory(TrajectoryBase):
    def eval(self, t: float):
        # Updatedtrajectory with smoother transitions
        position = np.array([0.5 * np.sin(t), 0.5 * np.cos(t), 0.2 * t])  # Smoother motion
        yaw_rate = 0.1  # Slower yaw rotation
        yaw = yaw_rate * t
        return TrajectoryState(t, position=position, yaw=yaw, yaw_rate=yaw_rate)

In [64]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d

In [65]:
#def run_and_render(initial_state: QuadrotorState, t_total: int = 2.0):
    #sim = SimulatorBase(
      #  dt=0.005,
       # dynamics=FullQuadrotorDynamics(),
       # controller=Controller(controller_p),    #Need to pass in the controller params
       # trajectory=JumpTrajectory(),
       # initial_state=initial_state,
       # t_total=t_total,
    #)
    
def run_and_render_smooth(initial_state: QuadrotorState, t_total: int = 2.0):
    sim = SimulatorBase(
        dt=0.005,
        dynamics=FullQuadrotorDynamics(),
        controller=Controller(controller_p),    # Use the same controller params
        trajectory=SmoothTrajectory(),          # Use smooth trajectory
        initial_state=initial_state,
        t_total=t_total,
    )
    
    output = sim.simulate()
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()


In [66]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

#Trying to dampen and slow response
controller_p.K_p = np.diag([0.5, 0.5, 50])
controller_p.K_d = np.diag([0.3, 0.3, 5])
controller_p.Ka_p = np.diag([150, 150, 25])
controller_p.Ka_d = np.diag([30, 30, 10])


# Run the simulation for 10 seconds
#run_and_render(initial_state, t_total=10.0)
run_and_render_smooth(initial_state, t_total=10.0)

TypeError: QuadrotorCommands.__init__() takes 2 positional arguments but 3 were given

### Looking at the controller behavior
If all is well, the quadrotor should follow the trajectory!

Try some experiments:
- Can you make the controller more/less damped and faster/slower to respond?
- Can you write a trajectory that has feasible velocities/positions?
    - Hint: the trajectory that's currently there is infeasible, why?

### Commit your updated notebook to your fork!